# Convolution Neural Network (CNN) - basic filters

Author: Chip Huyen

Jupyter scribe: Jiageng Liu

Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"

[https://cs20si.stanford.edu](cs20si.stanford.edu)

Simple examples of convolution to do some basic filters. Also demonstrates the use of TensorFlow data readers.<br>

We will use some popular filters for our image. It seems to be working with grayscale images, but not with rgb images. It's probably because I didn't choose the right kernels for rgb images.<br>

- kernels for rgb images have dimensions 3 x 3 x 3 x 3
- kernels for grayscale images have dimensions 3 x 3 x 1 x 1

_Note_:
When you call tf.train.string_input_producer, a tf.train.QueueRunner is added to the graph, which must be run using e.g. tf.train.start_queue_runners() else your session will run into deadlock and your program will crash.<br>

And to run QueueRunner, you need a coordinator to close to your queue for you. Without coordinator, your threads will keep on running outside session and you will have the error: _ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session_.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import sys
sys.path.append('..')

from matplotlib import gridspec as gridspec
from matplotlib import pyplot as plt
import tensorflow as tf

import kernels

FILENAME = 'data/friday.jpg'

## Read one image into TF

For demostration only. Easier to use Pillow in practice.

In [ ]:
filename_queue = tf.train.string_input_producer([FILENAME])
image_reader = tf.WholeFileReader()
_, image_file = image_reader.read(filename_queue)
image = tf.image.decode_jpeg(image_file, channels=3)
image = tf.cast(image, tf.float32) / 256.0 # cast to float to make conv2d work

## Convolve filter with the image

Preparation: make grayscale and batch

In [ ]:
image = tf.image.rgb_to_grayscale(image)
image = tf.expand_dims(image, 0)  # make it into a batch of 1 element

Define some constants. See lecture for details.

In [ ]:
kernels = [kernels.BLUR_FILTER, kernels.SHARPEN_FILTER, kernels.EDGE_FILTER, 
                    kernels.TOP_SOBEL, kernels.EMBOSS_FILTER]
strides = [1, 3, 3, 1]
padding = 'SAME'

Convolve with various filters.

In [ ]:
images = [image[0]]
for i, kernel in enumerate(kernels):
    filtered_image = tf.nn.conv2d(image, kernel, strides=strides, padding=padding)[0]
    if i == 2:
        filtered_image = tf.minimum(tf.nn.relu(filtered_image), 255)
    images.append(filtered_image)

## Session execution

In [ ]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    images = sess.run(images)
    coord.request_stop()
    coord.join(threads)

## Visualization

In [ ]:
gs = gridspec.GridSpec(1, len(images))
for i, image in enumerate(images):
    plt.subplot(gs[0, i])
    image = image.reshape(image.shape[0], image.shape[1])
    plt.imshow(image, cmap='gray')
    plt.axis('off')
plt.show()